In [1]:
import requests
from datetime import datetime

import os
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2 import service_account
from googleapiclient.discovery import build

def setup_gdrive(credentials_secret):
    with open('credentials.json', 'w') as f:
        f.write(os.environ[credentials_secret])
        
    # Load the credentials from the JSON key file
    credentials = service_account.Credentials.from_service_account_file(
        'credentials.json',
        scopes=['https://www.googleapis.com/auth/drive']
    )

    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service



In [2]:
with open('enre-414003-3724f64a117f.json') as f:
    credentials = f.read()

os.environ['GDRIVE_SECRET'] = credentials

In [3]:
drive_service = setup_gdrive('GDRIVE_SECRET')

In [4]:
import os

def download_file(file, path, skip_existing=False):
    file_id = file['id']
    request = drive_service.files().get_media(fileId=file_id)
    if skip_existing and os.path.exists(os.path.join(path, file['name'])):
        return False
    
    with open(os.path.join(path, file['name']), 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            return True

In [5]:
def download_all_files(path, skip_existing):
    response = drive_service.files().list().execute()
    
    files = response['files']
    page_no = 1
    while response.get('nextPageToken'):
        print('Downloading page', page_no)
        for file in files:
            download_file(file, path, skip_existing=skip_existing)
        
        response = drive_service.files().list(pageToken=response['nextPageToken']).execute()
        files = response['files']
        page_no += 1
        
    print('Finished Downloading')

In [ ]:
import os

if not os.path.exists('data'): os.makedirs('data')
    
download_all_files('data', skip_existing=True)